In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

Read the json file and return a dictionary

In [2]:
file_name = 'FE_dispatch_95_5_median_new.json'

with open(file_name) as f:
    sub_data = json.load(f)

Here are the keys of this dictionary.

In [3]:
sub_data.keys()

dict_keys(['cluster_95_dispatch', 'cluster_5_dispatch', 'median_dispatch'])

The 'cluster_95_dispatch' is the sub_scenario data for 95 quantile average capacity factor. 

The value of this key is still a dictionary

In [4]:
sub_95_data = sub_data['cluster_95_dispatch']

type(sub_95_data)

dict

In the 'sub_95_data' dictionary, the keys are the scenario number, from 0 to number of representative days -1.

Here the json will automatically set the keys in the dictionary to string instead of float.  

In [5]:
print(sub_95_data.keys())

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19'])


Here you can have the list of lenght 24, which is the time series capacity factor data.

In [6]:
print('The type of the 0.95 quantile average capacity factor data of scenario 0 is',type(sub_95_data['0']))

print('The length of the 0.95 quantile average capacity factor data of scenario 0 is',len(sub_95_data['0']))

The type of the 0.95 quantile average capacity factor data of scenario 0 is <class 'list'>
The length of the 0.95 quantile average capacity factor data of scenario 0 is 24


In [7]:
sub_95_data['0']

[[1.2000000000000002],
 [1.2000000000000002],
 [1.2000000000000002],
 [1.2000000000000002],
 [1.2000000000000002],
 [1.2000000000000002],
 [1.2000000000000002],
 [1.2000000000000002],
 [1.0],
 [0.8172056776315789],
 [1.0],
 [1.0],
 [1.0],
 [0.6052631578947368],
 [0.21052631578947367],
 [0.0],
 [0.39473684210526316],
 [0.0],
 [0.39473684210526316],
 [0.7894736842105263],
 [1.0],
 [1.0],
 [1.0],
 [1.0]]

Separate the capacity factor into generator capacity factor and storage capacity factor

In [11]:
sep_data = {}
for i in sub_data:
    sep_data[i] = {}
    for j in sub_data[i]:
        gen = []
        sto = []
        for t in sub_data[i][j]:
            if t[0] >= 1.0001:
                # if t[0] >= 1, then the storage is dispatched and the generator is working at full capacity
                sto.append((t[0]-1)/0.2)
                gen.append(1)
            else:
                # if t[0] <= 1, then the storage is not working and the generator is working at the given capacity
                sto.append(0)
                gen.append(t[0])
        sep_data[i][j] = [gen,sto]
sep_data.keys()

dict_keys(['cluster_95_dispatch', 'cluster_5_dispatch', 'median_dispatch'])

Save the data to json file.

In [10]:
with open('FE_dispatch_95_5_median_separate.json', 'w') as f:
    json.dump(sep_data, f)

In [16]:
gen_cf = sep_data['median_dispatch']['0'][0]
stor_cf = sep_data['median_dispatch']['0'][1]

P_ = 436 # MW
Pmin = 284 # MW
P_s = 15 # MW this should be a variable

for tg, ts in zip(gen_cf,stor_cf):
    P_out = (P_ - Pmin)*tg + Pmin + P_s*ts
    print(P_out)

338.010639
398.010639
436.0
436.0
436.0
436.0
436.0
436.0
429.407283
436.0
376.0
316.0
284.0
284.0
308.442448
368.442448
428.442448
436.0
436.0
436.0
436.0
436.0
436.0
436.0


In [23]:
# this for no sub scenario model.
full_day_gen_cf = [1]*24
full_day_stor_cf = [0]*24

zero_day_gen_cf = [0]*24
zero_day_stor_cf = [0]*24

num_rep_day = 20+2
cluster_cf = sep_data['median_dispatch']

for i in range(num_rep_day):
    if i == 0:
        gen_cf = zero_day_gen_cf
        stor_cf = zero_day_stor_cf
        print('zero_day')
    elif i == num_rep_day-1:
        gen_cf = full_day_gen_cf
        stor_cf = full_day_stor_cf
        print('full_day')
    else:
        scenario = str(i-1)
        gen_cf = cluster_cf[scenario][0]
        stor_cf = cluster_cf[scenario][1]
        print('day from clustering')
#     for t, blk in enumerate(blks): # len(blk) = 24 
#         blk.output_const = Constraint(expr = blk.fs.gen_elec_power[0] + blk_storage.elec_out[0] >= m.P_ - m.Pmin)*1000*gen_cf[t] + m.Pmin + m.P_s*stor_cf[t]
    scenario_model.total_cost = Expression(expr=scenario_model.dispatch_frequency*365*sum([blk.operate_total_cost for blk in blks]))

zero_day
day from clustering
day from clustering
day from clustering
day from clustering
day from clustering
day from clustering
day from clustering
day from clustering
day from clustering
day from clustering
day from clustering
day from clustering
day from clustering
day from clustering
day from clustering
day from clustering
day from clustering
day from clustering
day from clustering
day from clustering
full_day



sub_scenario_dispatch_profile = spe_data[keys][scenario_number]

keys in ['cluster_95_dispatch', 'cluster_5_dispatch', 'median_dispatch']

scenario_number in list(str(range(30)))

sub_scenario_dispatch_profile[0] = generator profile

sub_scenario_dispatch_profile[1] = storage profile

For the full capacity days and zero capacity days, each sub-scenario is the same.

full days:

Generator_capacity_profile_full = [1]*24

Storage_capacity_profile_full = [0]*24

zero days:

Generator_capacity_profile_full = [0]*24

Storage_capacity_profile_full = [0]*24